In [1]:
import os
%pwd

'/home/om/Desktop/dl-project/research'

In [2]:
os.chdir("/home/om/Desktop/dl-project")

In [3]:
%pwd

'/home/om/Desktop/dl-project'

In [ ]:
#enitty : return type of file

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Data Ingestion configuration.
    """
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_url=config.source_url,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config

In [7]:
import os
import zipfile
import gdown
from cnnClassifier.utils.common import get_size_of_file
from cnnClassifier import logger

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
        Downloads the dataset from the specified URL and saves it to the local directory.
        """
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading file from {self.config.source_url} to {self.config.local_data_file}")

            prefix = "https://drive.google.com/uc?/export=download&id="
            file_id = dataset_url.split("/")[-2]
            gdown.download(prefix+file_id, str(zip_download_dir), quiet=False)
            logger.info(f"Downloaded file size: {get_size_of_file(self.config.local_data_file)} bytes")
        except Exception as e:
            logger.error(f"Error occurred while downloading file: {e}")
            raise e

    def extract_zip_file(self):
        logger.info(f"Extracting zip file {self.config.local_data_file} to {self.config.unzip_dir}")
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        logger.info("Extraction completed")

In [17]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    # data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"An error occurred during data ingestion: {e}")
    raise e


2025-06-15 17:48:02,213 - INFO - common - Created directory: artifacts
2025-06-15 17:48:02,214 - INFO - common - Created directory: artifacts/data_ingestion
2025-06-15 17:48:02,215 - INFO - 2275616975 - Extracting zip file artifacts/data_ingestion/kidney-CT_Scan.zip to artifacts/data_ingestion


 12%|█▏        | 196M/1.63G [13:42<1:40:09, 238kB/s]


2025-06-15 17:48:12,637 - INFO - 2275616975 - Extraction completed
